In [1]:
import pandas as pd
from typing import Dict, List, Tuple
import json
from tqdm.auto import tqdm
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

In [2]:
parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv.parquet")
parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv.parquet")
parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv.parquet")

parquet_file_abcd_conv_test = Path("../data/ABCD_tripfiles_conv_test.parquet")
parquet_file_mnop_conv_test = Path("../data/MNOP_tripfiles_conv_test.parquet")
parquet_file_zyxw_conv_test = Path("../data/ZYXW_tripfiles_conv_test.parquet")

In [66]:
# ab_conv_test = pd.read_parquet(parquet_file_abcd_conv_test)
# mn_conv_test = pd.read_parquet(parquet_file_mnop_conv_test)
# zy_conv_test = pd.read_parquet(parquet_file_zyxw_conv_test)

In [67]:
ab_conv_test

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details,data_CreateLoadSheetAction
0,33902381,2024-05-01 09:14:42,AB,2359,1,BLR,service-acco,ASMMsgProcessor,"2024-05-01 09:14:42,420 INFO [3f326d134d9de45...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2...",None
1,33910427,2024-05-01 10:28:12,AB,2104,2,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,745 INFO [de24be75947007f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2...",None
2,33910380,2024-05-01 10:28:11,AB,2109,2,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:11,301 INFO [df3a1cc7d574161...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2...",None
3,33909790,2024-05-01 10:28:12,AB,2110,2,BOM,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,189 INFO [9e66dfb1ffe9048...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2...",None
4,33902706,2024-05-01 09:14:41,AB,2127,2,BOM,service-acco,ASMMsgProcessor,"2024-05-01 09:14:41,884 INFO [829566f527b588f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2...",None
...,...,...,...,...,...,...,...,...,...,...,...
709608,33875192,2024-05-01 04:07:01,AB,2372,30,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:07:01,675 INFO [90c6642a105b86f...",TOTAL Pax: 131 Y: 131 Jump: 0 StandBy: ...,None
709609,33875261,2024-05-01 04:08:01,AB,2372,30,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:08:01,279 INFO [76f51797d6d0b42...",com.onesystem.lc2.paxactuals.dto.PaxDataDTO [ ...,None
709610,33875264,2024-05-01 04:08:01,AB,2372,30,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:08:01,281 INFO [76f51797d6d0b42...",TOTAL Pax: 131 Y: 131 Jump: 0 StandBy: ...,None
709611,33875306,2024-05-01 04:09:01,AB,2372,30,GAU,service-acco,StorePaxDataAction,"2024-05-01 04:09:01,465 INFO [ef79e34fa812d85...",com.onesystem.lc2.paxactuals.dto.PaxDataDTO [ ...,None


In [3]:
ab_conv = pd.read_parquet(parquet_file_abcd_conv)
mn_conv = pd.read_parquet(parquet_file_mnop_conv)
zy_conv = pd.read_parquet(parquet_file_zyxw_conv)

In [24]:
class Weight:
    def __init__(self, key, description=None):
        self.key = key
        self.description = description

    def get_key(self):
        return self.key

    def get_description(self):
        return self.description

In [26]:
class WeightCluster:
    def __init__(self, name: str, weights: Dict[Weight, str]):
        self._name = name
        self._weights = weights

    def get_weight_cluster(self):
        return self._weights

In [25]:
class Action:
    def __init__(self, name: str, weights: Dict[Weight, str]):
        self._name = name
        self._weights = weights

    def get_name(self):
        return self._name

    def get_weights(self):
        return self._weights

    def get_weight_items(
        self,
    ):
        return self._weights.items()

In [28]:
EZFW = Weight("EZFW", "Estimated Zero Fuel Weight")
AZFW = Weight("AZFW", "Actual Zero Fuel Weight")
ETOW = Weight("ETOW", "Estimated Takeoff Weight")
ATOW = Weight("ATOW", "Actual Takeoff Weight")
TTL = Weight("TTL", "Total Traffic Load")
DOW = Weight("DOW", "Dry Operating Weight")
TOF = Weight("TOF", "Take Off Fuel")
TF = Weight("TF", "Trip Fuel")
ALW = Weight("ALW", "LANDING WEIGHT ACTUAL")

In [29]:
LOADSHEETACTION = {
    TTL: "TOTAL TRAFFIC LOAD",
    DOW: "DRY OPERATING WEIGHT",
    AZFW: "ZERO FUEL WEIGHT ACTUAL",
    TOF: "TAKE OFF FUEL",
    ATOW: "TAKE OFF WEIGHT ACTUAL",
    TF: "TRIP",
    ALW: "LANDING WEIGHT ACTUAL",
}

In [30]:
foo = [
    # Action(name="RampFinalAction", weights={EZFW: "EZFW"}), # EZFW is not a value just a status
    Action(name="CreateLoadsheetAction", weights=LOADSHEETACTION),
    Action(name="SendLoadsheetAction", weights=LOADSHEETACTION),
]

In [58]:
import numbers


maa = "17"
type(maa)
print(isinstance(maa, numbers.Number))
print(isinstance(maa, str))
print(maa.isdigit())
type(eval(maa))
# maa.isnumeric()
# maa.isdigit()

False
True
True


int

In [59]:
# Recursive function to find the value for a given key
def find_value(data: dict | list, key: str):
    if isinstance(data, dict):
        for k, v in data.items():
            if k == key:
                if not isinstance(v, numbers.Number):
                    if isinstance(v, str) and v.isdigit():
                        v = eval(v)
                    else:
                        raise ValueError("Value not a number string or a number")
                if v < 1_000_000:
                    return v
                return None
            else:
                found = find_value(v, key)
                if found is not None:
                    return found
    elif isinstance(data, list):
        for item in data:
            found = find_value(item, key)
            if found is not None:
                return found
    return None


# Function to apply the recursive search to JSON data
def extract_key(json_str: str, key: str):
    data = json.loads(json_str)
    return find_value(data, key)

In [60]:
df = mn_conv.copy()
df = df.replace({None: pd.NA})

In [61]:
for action in foo:
    for weight, key in action.get_weight_items():

        mask = (df.action_name == action.get_name()) & (
            ~df[f"data_{action.get_name()}"].isna()
        )

        # Use loc to update the DataFrame directly
        df.loc[mask, weight.get_key()] = df.loc[
            mask, f"data_{action.get_name()}"
        ].apply(lambda x: extract_key(x, key))

In [62]:
df

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details,...,data_UpdateFuelDataAction,data_UpdateLoadTableAction,data_StorePaxDataGuiAction,TTL,DOW,AZFW,TOF,ATOW,TF,ALW
0,137524484,2024-04-30 04:01:47,MN,1262,30,DUB,human,AssignLoadplanAction,"2024-04-30 04:01:47,383 INFO [a277234c22fa2e5...",com.systemone.lc2.loadplan.dto.PositionAssignm...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,137524940,2024-04-30 04:01:50,MN,1262,30,DUB,human,AssignLoadplanAction,"2024-04-30 04:01:50,188 INFO [3b152cbdf5b057e...",com.systemone.lc2.loadplan.dto.PositionAssignm...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,137524943,2024-04-30 04:01:50,MN,1262,30,DUB,human,AssignLoadplanAction,"2024-04-30 04:01:50,193 INFO [3b152cbdf5b057e...",com.systemone.lc2.manualloadplanning.dto.LoadD...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,137524964,2024-04-30 04:05:32,MN,1630,30,DUB,human,AssignLoadplanAction,"2024-04-30 04:05:32,214 INFO [52735a0dd84d57d...",com.systemone.lc2.loadplan.dto.PositionAssignm...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,137525021,2024-04-30 04:02:12,MN,1202,30,DUB,human,AssignLoadplanAction,"2024-04-30 04:02:12,081 INFO [8d65801e1dbb10e...",com.systemone.lc2.loadplan.dto.PositionAssignm...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108956,138586814,2024-05-07 00:14:36,MN,1088,6,MSP,service-acco,TransferCheckinDataAction,"2024-05-07 00:14:36,501 INFO [81001001d35136f...",TOTAL Pax: 180 Y: 172 J: 8 Jump: 0 Sta...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108957,138586940,2024-05-07 00:15:36,MN,1088,6,MSP,service-acco,TransferCheckinDataAction,"2024-05-07 00:15:36,449 INFO [358c06cbf3891e6...",DUB \r\n J ...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108958,138587005,2024-05-07 00:15:36,MN,1088,6,MSP,service-acco,TransferCheckinDataAction,"2024-05-07 00:15:36,455 INFO [358c06cbf3891e6...",TOTAL Pax: 180 Y: 172 J: 8 Jump: 0 Sta...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108959,138587020,2024-05-07 00:16:36,MN,1088,6,MSP,service-acco,TransferCheckinDataAction,"2024-05-07 00:16:36,274 INFO [410043c51bf5289...",DUB \r\n J ...,...,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# for name, keys in weights_to_extract.items():
#     print(name, keys)
#     for key, value in keys["keys"].items():
        